# 목표

1 지역 입력하면 기본적으로 4개 평균, 표준편차 보여주기

2 H 입력 받기

3 데이터 처리하고 점수 몇점 이내가 몇개 있는지 말해주기 / 몇개 출력할지 말해주기

4 df 출력하고 지도까지 출력하기

In [1]:
import pandas as pd
import numpy as np
import os
import glob #파일 한꺼번에 부르기 위해서 사용
import re

In [2]:
import folium

In [3]:
import seaborn as sns
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

plt.rc('font',family='Malgun Gothic') #한글 폰트 안깨지려면 이거 해야함
%matplotlib inline

In [4]:
data_frame=pd.read_excel('2018_modified_data_my.xlsx',endcoding="utf-8")

data_frame.head()

,시군구,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,거래금액(만원),건축년도,도로명,연도,월,SiDo,SiGunGu,DongLi,ID
0,강원도 강릉시 강동면 모전리,단독,8,64.20,205.0,201801,10520,2018,강동초교길,2018,1,강원도,강릉시,모전리(강동면),강릉
1,강원도 강릉시 견소동,단독,12,35.60,81.0,201801,4000,1990,창해로,2018,1,강원도,강릉시,견소동,강릉
2,강원도 강릉시 교동,단독,12,99.90,160.0,201801,16500,1982,원대로8번길,2018,1,강원도,강릉시,교동,강릉
3,강원도 강릉시 교동,단독,12,326.88,211.6,201801,64300,2005,율곡초교길47번길,2018,1,강원도,강릉시,교동,강릉
4,강원도 강릉시 교동,단독,8,154.99,240.2,201801,19000,1980,원대로20번길,2018,1,강원도,강릉시,교동,강릉


In [5]:
# 지역 
region= "서울특별시 연희동"
idx = (data_frame["SiDo"]==region.split(" ")[0]) & (data_frame["DongLi"]==region.split(" ")[1])

temp_df = data_frame[idx]
if len(temp_df)==0 : 
    print("집 없음")
    
h1=150
h2=100
h3=2010
h4=50_000

H=(h1,h2,h3,h4)

list_col=['연면적(㎡)','대지면적(㎡)','건축년도','거래금액(만원)']

difference=(temp_df[list_col]-H)/temp_df[list_col].std()
temp_df["distance"]=np.linalg.norm(difference, axis=1) #아래 메시지는 에러는 아니고 warning 메세지. 이렇게 하는 것보다 명령어로 column 추가하는게 더 낫다는 뜻

#temp_df["distance"]


/home/neulbo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
print ('지역을 입력해주세요 : (e.g. 서울특별시 서대문구 신촌동)')
#region=input()
region="서울특별시 서대문구 연희동"
sido=region.split(" ")[0]
sigugun=region.split(" ")[1]
dongli=region.split(" ")[2]

sido, sigugun, dongli

지역을 입력해주세요 : (e.g. 서울특별시 서대문구 신촌동)


('서울특별시', '서대문구', '연희동')

In [7]:
list_col=['연면적(㎡)','대지면적(㎡)','건축년도','거래금액(만원)']

idx = (data_frame["SiDo"]==sido) & (data_frame["DongLi"]==dongli)
temp_df=data_frame[idx]

print ("해당 지역 조건의 평균은 다음과 같습니다. :\n")
print (temp_df[list_col].mean(),"\n")
print ("표준편차는 다음과 같습니다. :\n")
print (temp_df[list_col].std())

해당 지역 조건의 평균은 다음과 같습니다. :

연면적(㎡)         178.336429
대지면적(㎡)        179.463265
건축년도          1982.795918
거래금액(만원)    101570.387755
dtype: float64 

표준편차는 다음과 같습니다. :

연면적(㎡)        159.729061
대지면적(㎡)       116.029273
건축년도           12.947920
거래금액(만원)    85699.986733
dtype: float64


* 연이자 4프로로 8.6억이면 월에 287만, 연면적 160이면 48.5평.

* 개인적으로 월 10만원에 5평 왔다갔다 하는 건 ok라고 생각함 (1/3 sigma ~ 3 sigma)

* 배관 및 전기 설비 수명이 20년인것을 감안하면, 월 10만원에 5년정도 왔다갔다 하는 건 ok라고 생각함 (1/3 sigma ~ 6 sigma)

* 위의 판단을 토대로 가중치를 임의로 주겠음

In [8]:
# 가중치 줘서 norm 구하는 것 test

H = [150,100,2010,50000]
a = [1,1,1/6,3]
temp_df2=temp_df[list_col]-H

temp_df2=temp_df2/temp_df2.std()
temp_df2=temp_df2*a

dist=np.linalg.norm(temp_df2, axis=1)

temp_df["distance"]=dist

/home/neulbo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [9]:
temp_df=temp_df.sort_values(["distance"])

In [10]:
# 가중치를 줘서 norm 구한 것
def function_2(H,a) :
    temp_df2=temp_df[list_col]-H
    temp_df2=temp_df2/temp_df2.std()
    temp_df2=temp_df2*a
    dist=np.linalg.norm(temp_df2, axis=1)
    temp_df["distance"]=dist
    result=temp_df.sort_values(["distance"])

    result.reset_index(inplace=True,drop=True)
    
    return(result)
    
    

In [11]:
# 일반 필터
def function_1(H,k=0.3) :
    idx1=(np.abs(temp_df['건축년도']-H[2])<=3)
    idx2=(np.abs(temp_df['거래금액(만원)']-H[3])<=0.3*H[3])
    
    temp_df1=temp_df[idx1&idx2]
    
    if len(temp_df1)==0 :
        print("해당 조건의 집이 없습니다.\n")
    else :
        print(f'{len(temp_df1)}개의 집이 있습니다.\n')
        x1=temp_df1["연면적(㎡)"]
        x2=temp_df1["대지면적(㎡)"]
    
        idx3 = (np.abs(x1-H[0])<k*x1.std())
        idx4=(np.abs(x2-H[1])<k*x2.std())
    
        result = temp_df1[idx3&idx4]

        print(f'연면적 평균={x1.mean()},대지면적 평균={x2.mean()}')
        print(f'연면적 표준편차={x1.std()},대지면적 표준편차={x2.std()}')
    
        return(result)

In [12]:
print ("조건을 입력해주세요. (e.g. 150,100,2010,50000)")
#h=input()
h="150,100,1990,50000"
H1=h.split(",")
H=H1.copy()
for i in list(range(len(H1))) :
    H[i]=int(H1[i])

result1=function_1(H,0.3)

print("해당 조건과 거의 동일한 집들 입니다 : ")
print(result1,"\n")

print ("해당 조건과 유사한 다른 집들을 보여드리겠습니다. 몇 개를 원합니까? : ")
#n=input()
n="10"
m=int(n)

a=[1,1,1/6,3]
#a=[1,1,1,1]

result2=function_2(H,a)
result2_final=result2[:m]

print(result2_final)

조건을 입력해주세요. (e.g. 150,100,2010,50000)
9개의 집이 있습니다.

연면적 평균=201.43,대지면적 평균=113.23333333333333
연면적 표준편차=84.28677120402703,대지면적 표준편차=58.32786641049027
해당 조건과 거의 동일한 집들 입니다 : 
                  시군구 주택유형  도로조건  연면적(㎡)  대지면적(㎡)    계약년월  거래금액(만원)  건축년도  \
55986  서울특별시 서대문구 연희동   단독     8  129.63     83.0  201807     43500  1993   
5167   서울특별시 서대문구 연희동   단독     8  159.09     86.0  201801     40500  1991   
71219  서울특별시 서대문구 연희동  다가구     8  125.01     86.0  201809     38000  1991   
30854  서울특별시 서대문구 연희동   단독     8  149.04     86.0  201804     36000  1992   
48324  서울특별시 서대문구 연희동   단독     8  132.76     86.0  201806     36000  1992   

           도로명    연도  월   SiDo SiGunGu DongLi      ID  distance  
55986     홍제천로  2018  7  서울특별시    서대문구    연희동  서울 서대문  0.370659  
5167      홍제천로  2018  1  서울특별시    서대문구    연희동  서울 서대문  0.433826  
71219     홍제천로  2018  9  서울특별시    서대문구    연희동  서울 서대문  0.524699  
30854     홍제천로  2018  4  서울특별시    서대문구    연희동  서울 서대문  0.555390  
48324  홍제천로2나길  2018  6  서울특별시    서

# 지도 그리기

In [13]:
result1.reset_index(inplace=True,drop=True)

In [14]:
result1

,시군구,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,거래금액(만원),건축년도,도로명,연도,월,SiDo,SiGunGu,DongLi,ID,distance
0,서울특별시 서대문구 연희동,단독,8,129.63,83.0,201807,43500,1993,홍제천로,2018,7,서울특별시,서대문구,연희동,서울 서대문,0.370659
1,서울특별시 서대문구 연희동,단독,8,159.09,86.0,201801,40500,1991,홍제천로,2018,1,서울특별시,서대문구,연희동,서울 서대문,0.433826
2,서울특별시 서대문구 연희동,다가구,8,125.01,86.0,201809,38000,1991,홍제천로,2018,9,서울특별시,서대문구,연희동,서울 서대문,0.524699
3,서울특별시 서대문구 연희동,단독,8,149.04,86.0,201804,36000,1992,홍제천로,2018,4,서울특별시,서대문구,연희동,서울 서대문,0.555390
4,서울특별시 서대문구 연희동,단독,8,132.76,86.0,201806,36000,1992,홍제천로2나길,2018,6,서울특별시,서대문구,연희동,서울 서대문,0.565749


In [15]:
result2_final

,시군구,주택유형,도로조건,연면적(㎡),대지면적(㎡),계약년월,거래금액(만원),건축년도,도로명,연도,월,SiDo,SiGunGu,DongLi,ID,distance
0,서울특별시 서대문구 연희동,단독,8,129.63,83.0,201807,43500,1993,홍제천로,2018,7,서울특별시,서대문구,연희동,서울 서대문,0.301653
1,서울특별시 서대문구 연희동,단독,8,105.13,102.0,201805,52700,1978,가좌로,2018,5,서울특별시,서대문구,연희동,서울 서대문,0.334667
2,서울특별시 서대문구 연희동,단독,8,117.33,83.0,201809,43700,1994,홍제천로2가길,2018,9,서울특별시,서대문구,연희동,서울 서대문,0.338509
3,서울특별시 서대문구 연희동,단독,8,159.09,86.0,201801,40500,1991,홍제천로,2018,1,서울특별시,서대문구,연희동,서울 서대문,0.358547
4,서울특별시 서대문구 연희동,단독,12,136.86,93.0,201803,60000,1982,홍연길,2018,3,서울특별시,서대문구,연희동,서울 서대문,0.378883
5,서울특별시 서대문구 연희동,다가구,8,125.01,86.0,201809,38000,1991,홍제천로,2018,9,서울특별시,서대문구,연희동,서울 서대문,0.464393
6,서울특별시 서대문구 연희동,단독,8,161.52,102.0,201803,38500,1970,홍제천로2길,2018,3,서울특별시,서대문구,연희동,서울 서대문,0.483565
7,서울특별시 서대문구 연희동,단독,8,149.04,86.0,201804,36000,1992,홍제천로,2018,4,서울특별시,서대문구,연희동,서울 서대문,0.505408
8,서울특별시 서대문구 연희동,단독,8,132.76,86.0,201806,36000,1992,홍제천로2나길,2018,6,서울특별시,서대문구,연희동,서울 서대문,0.516770
9,서울특별시 서대문구 연희동,다가구,8,147.08,136.0,201807,59500,1969,홍제천로2길,2018,7,서울특별시,서대문구,연희동,서울 서대문,0.529398


In [17]:
AA=data_frame=pd.read_excel('2018_loc_for_add_in_seoul.xlsx',endcoding="utf-8")

In [18]:
AA.head()

,도로주소,x좌표,y좌표,경도,위도
0,서울특별시 강남구 강남대로,958087.110153,1.944950e+06,127.025802,37.502866
1,서울특별시 강남구 강남대로100길,958293.189028,1.944846e+06,127.028140,37.501941
2,서울특별시 강남구 강남대로102길,958251.261594,1.944949e+06,127.027660,37.502868
3,서울특별시 강남구 강남대로106길,958186.231392,1.944984e+06,127.026922,37.503174
4,서울특별시 강남구 강남대로110길,958313.842824,1.945043e+06,127.028362,37.503714


In [19]:
map_pos1=[]

for i in list(range(len(result1))) :
    map_add=[]
    map_add.append(result1["SiDo"][i])
    map_add.append(result1["SiGunGu"][i])
    map_add.append(result1["도로명"][i])
    map_ad=' '.join(map_add)
    print(map_ad)
    idx=(AA['도로주소']==map_ad)
    a=[]
    a.append(float(AA['위도'][idx]))
    a.append(float(AA['경도'][idx]))
    
    map_pos1.append(a)
    

서울특별시 서대문구 홍제천로
서울특별시 서대문구 홍제천로
서울특별시 서대문구 홍제천로
서울특별시 서대문구 홍제천로
서울특별시 서대문구 홍제천로2나길


In [20]:
map_pos2=[]

for i in list(range(len(result2_final))) :
    map_add=[]
    map_add.append(result2_final["SiDo"][i])
    map_add.append(result2_final["SiGunGu"][i])
    map_add.append(result2_final["도로명"][i])
    map_ad=' '.join(map_add)
    print(map_ad)
    idx=(AA['도로주소']==map_ad)
    a=[]
    a.append(float(AA['위도'][idx]))
    a.append(float(AA['경도'][idx]))
    
    map_pos2.append(a)
    

서울특별시 서대문구 홍제천로
서울특별시 서대문구 가좌로
서울특별시 서대문구 홍제천로2가길
서울특별시 서대문구 홍제천로
서울특별시 서대문구 홍연길
서울특별시 서대문구 홍제천로
서울특별시 서대문구 홍제천로2길
서울특별시 서대문구 홍제천로
서울특별시 서대문구 홍제천로2나길
서울특별시 서대문구 홍제천로2길


In [21]:
result1['map_pos']=map_pos1
result2_final['map_pos']=map_pos2

/home/neulbo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
map_pos2

[[37.57251693891692, 126.9244185212296],
 [37.58059472128438, 126.9283317989286],
 [37.57056325202573, 126.9216134263325],
 [37.57251693891692, 126.9244185212296],
 [37.57610519042048, 126.9301066348881],
 [37.57251693891692, 126.9244185212296],
 [37.57094419645209, 126.9223158427131],
 [37.57251693891692, 126.9244185212296],
 [37.57051863569244, 126.9219462496028],
 [37.57094419645209, 126.9223158427131]]

In [23]:
map_pos_pos=map_pos1+map_pos2
map_pos_pos

[[37.57251693891692, 126.9244185212296],
 [37.57251693891692, 126.9244185212296],
 [37.57251693891692, 126.9244185212296],
 [37.57251693891692, 126.9244185212296],
 [37.57051863569244, 126.9219462496028],
 [37.57251693891692, 126.9244185212296],
 [37.58059472128438, 126.9283317989286],
 [37.57056325202573, 126.9216134263325],
 [37.57251693891692, 126.9244185212296],
 [37.57610519042048, 126.9301066348881],
 [37.57251693891692, 126.9244185212296],
 [37.57094419645209, 126.9223158427131],
 [37.57251693891692, 126.9244185212296],
 [37.57051863569244, 126.9219462496028],
 [37.57094419645209, 126.9223158427131]]

In [24]:
#도로명 주소의 앞부분이 동일해서 마커가 겹치는 것 방지

for i in list(range(len(map_pos_pos))) :
    for j in list(range(len(map_pos_pos)))[i+1:len(map_pos_pos)] :
        if map_pos_pos[i] == map_pos_pos[j] :
            if j < len(map_pos1) :
                map_pos1[j][0] += 0.0001
                map_pos1[j][1] += 0.0001
            else :
                map_pos2[j-len(map_pos1)][0] += 0.0001
                map_pos2[j-len(map_pos1)][1] += 0.0001

In [25]:
map_pos1

[[37.57251693891692, 126.9244185212296],
 [37.57261693891692, 126.9245185212296],
 [37.572716938916926, 126.9246185212296],
 [37.57281693891693, 126.92471852122961],
 [37.57051863569244, 126.9219462496028]]

In [26]:
target_pos=map_pos2[0]

m=folium.Map(location=target_pos,min_zoom=15, max_zoom=20)

folium.Circle(location=target_pos,radius=1000, color='green', fill_color='green').add_to(m)

print (len(map_pos1))
print (len(map_pos2))

for i in list(range(len(map_pos1))) :
    j=str(i)
    name='집1-'+j
    folium.CircleMarker(location=map_pos1[i],radius=15, color='red', fill_color='red').add_to(m) 
    folium.Marker(map_pos1[i],popup=name,icon=folium.Icon(color='red')).add_to(m)

for i in list(range(len(map_pos2))) :
    j=str(i)
    name='집2-'+j
    folium.CircleMarker(location=map_pos2[i],radius=15, color='blue', fill_color='blue').add_to(m) 
    folium.Marker(map_pos2[i],popup=name,icon=folium.Icon(color='blue')).add_to(m)

    
m

5
10
